## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBSSBSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
710
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|           res_mean|    station|  total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+------+--------------------+------+
|-1878.4708578337422|232.81818128124905|-197.53695949174033|   1911.1|SBSSBSBB| 41.6628|-111.1861|       SNWD|0.09549059707354036|0.08159625628162587|0.07159393289397907|0.3651

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [15]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00427165,2008.0,-1878.470858
1,USC00426869,1982.0,-4874.616752
2,USC00426869,2010.0,-3781.859129
3,USC00426869,1973.0,-5465.200199
4,USC00426869,1971.0,-3816.684079


In [16]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00100803,USC00103732,USC00105275,USC00106053,USC00107346,USC00107353,USC00420487,USC00420924,USC00420928,USC00421590,...,USC00427271,USC00427318,USC00428828,USC00428885,USC00429346,USC00429595,USC00480915,USW00024101,USW00024120,USW00024126
year,,,,,,,,,,,,,,,,,,,,,
1929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,355.368726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931.0,NaN,NaN,NaN,512.620268,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,71.089235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932.0,NaN,NaN,NaN,-2074.531608,NaN,NaN,NaN,NaN,NaN,NaN,...,323.065300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1933.0,NaN,NaN,NaN,-1908.391837,NaN,NaN,NaN,NaN,NaN,NaN,...,21.665291,174.120609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935.0,NaN,NaN,-132.093943,-971.596227,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,-1865.482039,-2613.415178,NaN,NaN,NaN,NaN,NaN,NaN,...,231.428212,329.691520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1937.0,NaN,NaN,-1598.064241,-877.329506,NaN,NaN,NaN,NaN,NaN,NaN,...,402.866370,-176.787395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1938.0,NaN,NaN,657.388272,382.325529,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [9]:
def MS(Mat):
    return np.nanmean(Mat**2)

In [17]:
# coeff1
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  2379089.0691
MS removing mean-by-station=  1361932.2784 , fraction explained =  42.7540441387
MS removing mean-by-year   =  1396429.89226 , fraction explained =  41.3040095724


In [14]:
# coeff3
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  357987.817788
MS removing mean-by-station=  297067.574754 , fraction explained =  17.0174067404
MS removing mean-by-year   =  155071.1418 , fraction explained =  56.6825645749


In [10]:
# coeff2
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station), \
        ', fraction explained = ', (1- MS(tbl_minus_station) /MS(year_station_table))*100
print 'MS removing mean-by-year   = ',MS(tbl_minus_year), \
        ', fraction explained = ', (1- MS(tbl_minus_year) /MS(year_station_table)) * 100

total MS                   =  362293.993716
MS removing mean-by-station=  330969.764203 , fraction explained =  8.64608027064
MS removing mean-by-year   =  119749.511364 , fraction explained =  66.9468681676


In [11]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 601.90862572
0 after removing mean by year    = 346.048423438
0 after removing mean by stations= 331.033017742
1 after removing mean by year    = 329.728050024
1 after removing mean by stations= 329.52247672
2 after removing mean by year    = 329.479740895
2 after removing mean by stations= 329.468656752
3 after removing mean by year    = 329.465154481
3 after removing mean by stations= 329.463861302
4 after removing mean by year    = 329.463330679
4 after removing mean by stations= 329.463098636
